In [2]:
!pip --default-timeout=100 install -q ollama langchain langchain-community langchain-core pypdf reportlab lancedb beautifulsoup4 unstructured phidata


In [ ]:
# Cell 1: Imports and Setup (UPDATED)

import os
import json
from IPython.display import display, Markdown

# LangChain components for LLM and RAG
# from langchain_community.llms import Ollama as LCOllama <-- OLD
# from langchain_ollama import Ollama as LCOllama <-- Incorrect Name
from langchain_ollama import OllamaLLM as LCOllama # <-- NEW: Corrected Class Name import
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document as LangChainDocument
import uuid  
import pyarrow as pa 
# phi-agent components
from phi.model.ollama import Ollama as PhiOllama
from phi.vectordb.lancedb import LanceDb, SearchType
from phi.embedder.ollama import OllamaEmbedder as PhiOllamaEmbedder
from phi.document import Document as PhiDocument

# PDF processing
import PyPDF2

# For creating a dummy PDF (optional)
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas as r_canvas

print("All libraries imported successfully!")

# Cell 2: Ollama Model Initialization (No Changes)

model_id = "gemma:2b"
# Initialize Ollama model for phi-agent
phi_ollama_model = PhiOllama(id=model_id)
# Initialize Ollama embedder for phi-agent's vector DB
phi_ollama_embedder = PhiOllamaEmbedder(model=model_id)

print(f"Ollama model '{model_id}' initialized for LLM and Embedder.")


# Cell 3: PDF Text Extraction Function (No Changes)

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extracts text from a given PDF file using PyPDF2.
    """
    try:
        with open(pdf_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() or ""
            return text
    except Exception as e:
        print(f"Error extracting text from PDF: {e}")
        return ""

print("PDF extraction function defined.")


def process_invoice_data(pdf_path: str) -> list[PhiDocument]:
    """
    Extracts text from PDF and returns properly formatted PhiDocuments
    """
    print(f"Processing PDF: {pdf_path}")
    text = extract_text_from_pdf(pdf_path)
    if not text:
        print("No text extracted from PDF.")
        return []

    # Initialize the text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    
    # Split text and create PhiDocuments
    text_chunks = text_splitter.split_text(text)
    phi_chunks = [
        PhiDocument(
            name=f"chunk_{i}",
            content=chunk,  # Using 'content' consistently
            meta={"source": pdf_path}
        )
        for i, chunk in enumerate(text_chunks)
    ]
    
    print(f"Split PDF into {len(phi_chunks)} chunks.")
    return phi_chunks
print("Invoice data processing function defined.")


# Cell 5: Setup Vector Database (LanceDB) (No Changes)

def setup_vector_db(chunks: list[PhiDocument]) -> LanceDb:
    """
    More robust database setup with proper cleanup
    """
    import lancedb
    
    db_uri = "tmp/lancedb_invoices"
    
    # Clean up any existing database
    if os.path.exists(db_uri):
        try:
            import shutil
            shutil.rmtree(db_uri)
        except Exception as e:
            print(f"Warning: Could not clean database directory: {e}")
    
    os.makedirs(db_uri, exist_ok=True)
    
    # Connect to LanceDB with retry logic
    max_retries = 3
    for attempt in range(max_retries):
        try:
            db = lancedb.connect(db_uri)
            
            # Get embedding dimension
            test_embedding = phi_ollama_embedder.get_embedding("test")
            dim = len(test_embedding)
            print(f"Using embedding dimension: {dim}")
            
            # Define schema
            schema = pa.schema([
                pa.field("id", pa.string()),
                pa.field("content", pa.string()),
                pa.field("vector", pa.list_(pa.float32(), dim)),
                pa.field("metadata", pa.string())
            ])
            
            # Create table
            table = db.create_table("invoice_data", schema=schema)
            print("Created new table with proper schema")
            
            # Insert documents with batch processing
            if chunks:
                data = []
                for chunk in chunks:
                    try:
                        embedding = phi_ollama_embedder.get_embedding(chunk.content)
                        data.append({
                            "id": str(uuid.uuid4()),
                            "content": chunk.content,
                            "vector": embedding,
                            "metadata": json.dumps(chunk.meta if hasattr(chunk, 'meta') else {})
                        })
                    except Exception as e:
                        print(f"Error processing chunk: {e}")
                        continue
                
                if data:
                    # Insert in batches to prevent timeouts
                    batch_size = 5
                    for i in range(0, len(data), batch_size):
                        batch = data[i:i + batch_size]
                        table.add(batch)
                    print(f"Successfully inserted {len(data)} chunks")
                else:
                    print("No valid chunks to insert")
            
            return table
            
        except Exception as e:
            if attempt == max_retries - 1:
                raise
            print(f"Attempt {attempt + 1} failed, retrying... Error: {e}")
            time.sleep(1)  # Add delay between retries
# Cell 6: Define RAG Prompt (No Changes)

rag_prompt_template = ChatPromptTemplate.from_messages([
    ("system", (
        "You are an expert AI assistant specialized in extracting precise details from invoice documents. "
        "Your goal is to accurately answer questions based *only* on the provided invoice context. "
        "If the information is not present in the context, state that you cannot find it. "
        "For all extractions, return the response in a clear JSON format with appropriate keys for the extracted data."
        "\n---CONTEXT---\n"
        "{context}"
    )),
    ("human", "Query: {query}")
])

print("RAG prompt template defined.")

# Cell 7: Main Execution Flow (UPDATED)
# Cell 7: Main Execution Flow (Updated)
def main():
    # Setup: Create 'tmp' folder and define invoice PDF path
    pdf_dir = os.path.join(os.getcwd(), "tmp")
    os.makedirs(pdf_dir, exist_ok=True)
    pdf_path = os.path.join(pdf_dir, "invoice.pdf")

    # Create a dummy invoice if it doesn't exist
    if not os.path.exists(pdf_path):
        print("Creating a dummy 'invoice.pdf' for demonstration.")
        c = r_canvas.Canvas(pdf_path, pagesize=letter)
        c.drawString(100, 750, "Invoice No: INV-2023-001")
        c.drawString(100, 730, "Invoice Date: October 26, 2023")
        c.drawString(100, 710, "Customer: ABC Corp")
        c.drawString(100, 690, "Total Amount Due: $1234.56 USD")
        c.drawString(100, 670, "Payment Terms: Net 30")
        c.drawString(100, 650, "Line Item 1: Product X - Quantity: 2 - Price: $300.00 - Total: $600.00")
        c.drawString(100, 630, "Line Item 2: Service Y - Quantity: 1 - Price: $634.56 - Total: $634.56")
        c.save()
    else:
        print(f"Using existing invoice.pdf at: {pdf_path}")

    # Process PDF and setup the vector database
    chunks = process_invoice_data(pdf_path)
    if not chunks:
        print("Exiting: No text found in PDF.")
        return None
    
    vector_db_instance = setup_vector_db(chunks)
    return vector_db_instance

def query_invoice_data(vector_db: LanceDb, question: str):
    """
    Query the invoice data using the vector database
    """
    try:
        # Generate embedding for the question
        query_embedding = phi_ollama_embedder.get_embedding(question)
        
        # Perform the search (correct LanceDB syntax)
        results = vector_db.search(query_embedding).limit(3).to_list()
        
        # Combine the context from top results
        context = "\n".join([result["content"] for result in results])
        
        # Setup LLM chain
        llm = LCOllama(model=model_id)
        chain = rag_prompt_template | llm | StrOutputParser()
        
        # Get answer
        response = chain.invoke({
            "context": context,
            "query": question
        })
        
        display(Markdown(f"**Question:** {question}"))
        display(Markdown(f"**Answer:** {response}"))
    except Exception as e:
        print(f"Error querying data: {e}")

if __name__ == "__main__":
    # Get the vector db instance
    vector_db = main()
    
    # Only run queries if we have a valid vector db
    if vector_db:
        print("\nInvoice Query System Ready!")
        query_invoice_data(vector_db, "What is the total amount due for this invoice?")
        query_invoice_data(vector_db, "Who is the customer and what is their address?")
        query_invoice_data(vector_db, "What is the Invoice Number and Date?")
        query_invoice_data(vector_db, "What are the payment terms?")
        query_invoice_data(vector_db, "Who is the salesperson mentioned on the invoice?",)
        query_invoice_data(vector_db, "What is the description of the item purchased and its quantity?")
        query_invoice_data(vector_db,"What is the unit price of the decorative clay pottery?")
        query_invoice_data(vector_db, "What is the subtotal before taxes and shipping?")
        query_invoice_data(vector_db,"How much is the sales tax and shipping & handling?")
        query_invoice_data(vector_db, "Are there any special instructions or comments about the shipment?")
        query_invoice_data(vector_db,  "What is the phone number of Pottery & Co.?")
        
        
    else:
        print("Failed to initialize vector database")

All libraries imported successfully!
Ollama model 'gemma:2b' initialized for LLM and Embedder.
PDF extraction function defined.
Invoice data processing function defined.
RAG prompt template defined.
Using existing invoice.pdf at: C:\Users\ALISH\Agent Practice\tmp\invoice.pdf
Processing PDF: C:\Users\ALISH\Agent Practice\tmp\invoice.pdf
Split PDF into 2 chunks.
Using embedding dimension: 2048
Created new table with proper schema
Successfully inserted 2 chunks

Invoice Query System Ready!


**Question:** What is the total amount due for this invoice?

**Answer:** {
  "total_due": 1389.99
}

**Question:** Who is the customer and what is their address?

**Answer:** {
  "customer_name": "Mollie Grau",
  "customer_address": "210 Stars Avenue, Berkeley, CA 78910"
}

**Question:** What is the Invoice Number and Date?

**Answer:** {
  "invoice_number": "100",
  "date": "1/1/23"
}